合并同一首歌的多行（多艺人/多国籍、多日期）

7 个音频特征做 Min-Max 归一化

排名取该歌历史最优名次（最小 Rank）

上榜天数= 该歌在不同日期出现的去重天数

最后只保留：id, Title, Artists, Nationality, 7个音频特征, Rank, DaysOnChart 

In [9]:
# -*- coding: utf-8 -*-
"""
预处理说明：
- 输入：原始 CSV，以 ';' 分隔，包含每日榜单前200（2017/01/01 ~ 2023/05/31）
- 目标：合并到歌曲级（id），保留：
  id, Title, Artists(去重拼接), Nationality(去重拼接),
  7个音频特征(均值后再MinMax归一化), Rank(历史最优名次), DaysOnChart(上榜去重天数)
- 进度：对耗时的 groupby-apply 加 tqdm 进度条
"""

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

# ========= 可配置路径 =========
SRC = Path(r"C:\Users\Akari\OneDrive\Desktop\SEM 1\AML\Final CW\Datas\Spotify_Dataset_V3.csv")         # 原始CSV（;分隔）
OUT = Path(r"C:\Users\Akari\OneDrive\Desktop\SEM 1\AML\Final CW\Datas\spotify_preprocess_dayAndRank.csv")         # 输出CSV

# ========= 工具函数 =========
def split_artists(s: str):
    """把 Artists 列里可能出现的制表符/逗号等分隔统一拆分为列表。"""
    if pd.isna(s):
        return []
    s = str(s).replace('\t', ',').replace('  ', ' ')
    parts = [p.strip() for p in s.replace(' ,', ',').split(',') if p.strip()]
    return parts

def uniq_preserve_order(seq):
    """去重并保序。"""
    return list(dict.fromkeys(seq))

# tqdm 设置（支持 groupby 的 progress_apply）
tqdm.pandas(desc="Processing groups")

# ========= Step 0: 读取 =========
df = pd.read_csv(SRC, delimiter=';', dtype=str)

# 转数值列
num_cols = ['Danceability','Energy','Loudness','Speechiness','Acousticness','Instrumentalness','Valence','Rank']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# 解析日期
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y", errors='coerce')

# 基础清理
df = df.dropna(subset=['id'])
df['Title'] = df['Title'].fillna('')

# 存在性探测
has_single_artist = 'Artist (Ind.)' in df.columns
has_artists       = 'Artists' in df.columns
has_single_nat    = 'Nationality' in df.columns

# ========= Step 1: DaysOnChart（按 id-日期 去重后计数）=========
if 'Date' in df.columns:
    # 每日同一首歌可能多行（多艺人），去重到 (id, Date)
    days_per_id = (df.dropna(subset=['Date'])
                     .drop_duplicates(subset=['id', 'Date'])
                     .groupby('id')['Date']
                     .nunique()
                     .rename('DaysOnChart')
                     .astype(int))
else:
    days_per_id = pd.Series(0, index=df['id'].unique(), name='DaysOnChart').astype(int)

# ========= Step 2: Rank（历史最优名次）=========
peak_rank = (df.dropna(subset=['Rank'])
               .groupby('id')['Rank']
               .min()
               .rename('Rank'))

# ========= Step 3: Artists / Nationality 聚合（无弃用警告 + 进度条）=========
# 说明：使用 groupby 后“显式选择列”再 apply，避免 DataFrameGroupBy.apply 的弃用警告

# 3.1 艺人集合
if has_single_artist and has_artists:
    # 同时利用单艺人列 & 多艺人拼接列
    grp = df.groupby('id')[['Artists', 'Artist (Ind.)']]
    def collect_artists(g):
        single = [a for a in g['Artist (Ind.)'].astype(str) if pd.notna(a) and a.strip()]
        multi  = sum([split_artists(x) for x in g['Artists'].astype(str)], [])
        # 去空格（名字内部空格去掉保持一致性，可按需调整）
        merged = [s.replace(' ', '') for s in (single + multi) if s.strip()]
        return uniq_preserve_order(merged)
    artists_per_id = grp.progress_apply(collect_artists).rename('Artists')

elif has_single_artist:
    grp = df.groupby('id')[['Artist (Ind.)']]
    def collect_artists_single(g):
        single = [a for a in g['Artist (Ind.)'].astype(str) if pd.notna(a) and a.strip()]
        merged = [s.replace(' ', '') for s in single]
        return uniq_preserve_order(merged)
    artists_per_id = grp.progress_apply(collect_artists_single).rename('Artists')

elif has_artists:
    grp = df.groupby('id')[['Artists']]
    def collect_artists_multi(g):
        multi  = sum([split_artists(x) for x in g['Artists'].astype(str)], [])
        merged = [s.replace(' ', '') for s in multi]
        return uniq_preserve_order(merged)
    artists_per_id = grp.progress_apply(collect_artists_multi).rename('Artists')

else:
    # 都没有，则为空
    artists_per_id = pd.Series([[]]*df['id'].nunique(),
                               index=df['id'].drop_duplicates().tolist(),
                               name='Artists')

# 3.2 国籍集合
if has_single_nat:
    grp_nat = df.groupby('id')[['Nationality']]
    def collect_nat(g):
        vals = [n for n in g['Nationality'].astype(str) if pd.notna(n) and n.strip()]
        return uniq_preserve_order(vals)
    nationality_per_id = grp_nat.progress_apply(collect_nat).rename('Nationality')
else:
    nationality_per_id = pd.Series([[]]*df['id'].nunique(),
                                   index=df['id'].drop_duplicates().tolist(),
                                   name='Nationality')

# ========= Step 4: 数值特征均值（按 id）=========
agg_num_cols = ['Danceability','Energy','Valence','Loudness','Speechiness','Acousticness','Instrumentalness']
num_mean = df.groupby('id')[agg_num_cols].mean()

# ========= Step 5: 标题取第一条 =========
title_first = df.groupby('id')['Title'].first()

# ========= Step 6: 汇总到歌曲级 =========
grouped = pd.concat([title_first, artists_per_id, nationality_per_id, num_mean], axis=1).reset_index()

# 字段串联（最终存成字符串，Artists 去掉内部空格、空格分隔；Nationality 保留原文本，空格分隔）
grouped['Artists'] = grouped['Artists'].apply(lambda lst: ', '.join(lst) if isinstance(lst, list) else '')
grouped['Nationality'] = grouped['Nationality'].apply(lambda lst: ', '.join(lst) if isinstance(lst, list) else '')

# ========= Step 7: Min-Max 归一化 =========
scaler = MinMaxScaler(feature_range=(0, 1))
grouped[agg_num_cols] = scaler.fit_transform(grouped[agg_num_cols])

# ========= Step 8: 合并 Rank / DaysOnChart =========
grouped = (grouped
           .merge(peak_rank, on='id', how='left')
           .merge(days_per_id, on='id', how='left'))

# 缺失兜底
if grouped['Rank'].isna().any():
    # 若全空则置0；否则用已知最大值兜底
    fallback = grouped['Rank'].max() if grouped['Rank'].notna().any() else 0
    grouped['Rank'] = grouped['Rank'].fillna(fallback)
grouped['Rank'] = grouped['Rank'].astype(int, errors='ignore')
grouped['DaysOnChart'] = grouped['DaysOnChart'].fillna(0).astype(int)

# ========= Step 9: 输出 =========
final_cols = [
    'id', 'Title', 'Artists', 'Nationality',
    'Danceability','Energy','Valence','Loudness','Speechiness','Acousticness','Instrumentalness',
    'Rank','DaysOnChart'
]
final_df = grouped[final_cols].copy()

final_df.to_csv(OUT, index=False, encoding='utf-8')
print(f"✅ 预处理完成：{OUT}  共 {len(final_df)} 首歌")


Processing groups: 100%|█████████████████████████████████████████████████████████| 9161/9161 [00:01<00:00, 7826.79it/s]


✅ 预处理完成：C:\Users\Akari\OneDrive\Desktop\SEM 1\AML\Final CW\Datas\spotify_preprocess_dayAndRank.csv  共 9161 首歌


将数据集分成训练集和测试集